노드에 여러 작업을 지시

In [4]:
# 이제 구성된 노드에 거래내역을 저장하고, PoW를 통해 블록을 생성하고, 구성된 데이터를 조회하는 등 블록체인 노드를 운영해보자!

import requests
import json
import pandas as pd 
import hashlib
import random


# 블록 조회
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5001/chain", headers=headers)

# 응답 내용과 상태 코드 확인
print("Status Code:", res.status_code)
print("Response Content:", res.text)

# 유효한 JSON 응답인 경우에만 JSON으로 변환
if res.headers['Content-Type'] == 'application/json' and res.status_code == 200:
    data = json.loads(res.text) #원래 책에는 저기 블록 조회에 json.loads(res.content)만 있었음
    print(data)
else:
    print("Invalid response or non-JSON content received")



Status Code: 404
Response Content: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

Invalid response or non-JSON content received


In [ ]:
# 트랜잭션 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender": "test_from",
    "recipient": "test_to",
    "amount" : 3,
}
requests.post("http://localhost:5001/transactions/new", headers=headers, data=json.dumps(data)).content

In [ ]:
# 채굴 명령 
headers = {'Content-Type: application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res) # response status가 출력됨
print(res.text) # 블록체인 내용이 출력됨

In [ ]:
# Pandas를 활용한 거래 내역 확인
status_json = json.loads(res.text)
status_json['chain']
tx_amount_l = []
tx_sender_l = []
tx_reciv_l = []
tx_time_l = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']

for each_tx in range(len(chain_tx)):
    tx_amount_l.append(chain_tx[each_tx]['amount'])
    tx_sender_l.append(chain_tx[each_tx]['sender'])
    tx_reciv_l.append(chain_tx[each_tx]['recipient'])
    tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l
df_tx['sender'] = tx_sender_l
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l
df_tx

In [ ]:
# 지갑별 잔액 계산 (계정별 잔고 조회 - user, received_amount, sended_amount, balance)
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status